# **Dependencies**

In [3]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from typing import Dict, List, Optional
from datetime import datetime, timedelta

ModuleNotFoundError: No module named 'pandas'

In [5]:
data_path = "/Users/dylantadas/Desktop/spring 25/i494- capstone project/dataset/"  # path to OULAD.zip

In [6]:
import zipfile
with zipfile.ZipFile(os.path.join(data_path, "OULAD.zip"), 'r') as zip_ref:
    zip_ref.extractall(data_path)

# **I. Data Processing**

## **Load Dataset**

In [7]:
# define utilized columns
demographic_cols = [
    'id_student', 'code_module', 'code_presentation',  # identifiers
    'gender', 'region', 'highest_education', 'imd_band', 'age_band',  # categorical features
    'num_of_prev_attempts', 'studied_credits', 'disability'  # mumerical features
]

vle_cols = [
    'id_student', 'code_module', 'code_presentation',  # identifiers
    'date', 'sum_click',  # activity metrics
    'activity_type'  # for feature creation
]

In [8]:
def load_raw_datasets() -> Dict[str, pd.DataFrame]:
    """Loads all CSV files from OULAD dataset with optimized memory usage."""

    # define column types to reduce memory
    dtype_dict = {
        'id_student': 'int32',
        'code_module': 'category',
        'code_presentation': 'category',
        'gender': 'category',
        'region': 'category',
        'highest_education': 'category',
        'imd_band': 'category',
        'age_band': 'category',
        'num_of_prev_attempts': 'int8',
        'disability': 'category',
        'date': 'int16',
        'sum_click': 'int16',
        'module_presentation_length': 'int16'
    }

    # define which columns needed from each file
    dataset_files = {
        'student_info': {
            'file': 'studentInfo.csv',
            'columns': [
                'id_student', 'code_module', 'code_presentation',
                'gender', 'region', 'highest_education', 'imd_band', 'age_band',
                'num_of_prev_attempts', 'studied_credits', 'disability'
            ]
        },
        'vle_interactions': {
            'file': 'studentVle.csv',
            'columns': [
                'id_student', 'code_module', 'code_presentation',
                'date', 'sum_click', 'id_site'
            ]
        },
        'vle_materials': {
            'file': 'vle.csv',
            'columns': None
        },
        'assessments': {
            'file': 'assessments.csv',
            'columns': None
        },
        'student_assessments': {
            'file': 'studentAssessment.csv',
            'columns': None
        },
        'courses': {
            'file': 'courses.csv',
            'columns': [
                'code_module',
                'code_presentation',
                'module_presentation_length'
            ]
        }
    }

    datasets = {}
    total_rows = 0

    for key, file_info in dataset_files.items():
        try:
            # load csw with specified columns and optimized dtypes
            df = pd.read_csv(
                    os.path.join(data_path, file_info['file']),
                    usecols=file_info['columns'],
                    dtype={col: dtype_dict.get(col) for col in (file_info['columns'] or [])
                           if col in dtype_dict}
            )
            
            total_rows += len(df)
            print(f"Loaded {file_info['file']}: {len(df)} rows, {len(df.columns)} columns")
            datasets[key] = df

        except FileNotFoundError:
            print(f"Error: {file_info['file']} not found")
            datasets[key] = None

        except ValueError as e:
            print(f"Error loading {file_info['file']}: {str(e)}")
            print("Loading all columns instead...")
            # fallback: load all columns if unmatched specified columns
            df = pd.read_csv(file_info['file'])
            total_rows += len(df)
            print(f"Loaded {file_info['file']}: {len(df)} rows, {len(df.columns)} columns")
            datasets[key] = df

    print(f"\nTotal rows loaded across all files: {total_rows:,}")
    return datasets

## **Exploratory Data Analysis**

In [9]:
def perform_automated_eda(datasets):
    """Performs automated EDA and displays results directly in console, identifying potential data quality issues."""

    for name, df in datasets.items():
        print(f"\n{'='*50}")
        print(f"Analysis of {name}")
        print(f"{'='*50}")

        # dataset overview
        print("\nDataset Overview:")
        print(f"Number of rows: {len(df)}")
        print(f"Number of columns: {len(df.columns)}")
        print("\nColumns and their data types:")
        print(df.dtypes)

        # missing values
        print("\nMissing Values Analysis:")
        missing = df.isnull().sum()
        if missing.any():
            print(missing[missing > 0])
        else:
            print("No missing values found")

        # numerical analysis
        numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
        if len(numerical_cols) > 0:
            print("\nNumerical Columns Summary:")
            print(df[numerical_cols].describe())

        # categorical analysis
        categorical_cols = df.select_dtypes(include=['object']).columns
        if len(categorical_cols) > 0:
            print("\nCategorical Columns Summary:")
            for col in categorical_cols:
                print(f"\nDistribution of {col}:")
                print(df[col].value_counts(normalize=True).head())

In [10]:
def analyze_student_performance(datasets):
    """Analyzes student performance patterns, displays demographic-related assessment score insights."""

    student_data = datasets['student_info'].merge(
        datasets['student_assessments'],
        on='id_student'
    )

    print("\nStudent Performance Analysis")
    print("="*50)

    # overall score distribution
    print("\nScore Distribution:")
    print(student_data['score'].describe())

    # performance by demographic groups
    for column in ['gender', 'age_band', 'imd_band']:
        print(f"\nAverage Score by {column}:")
        print(student_data.groupby(column, observed=False)['score'].mean())

In [11]:
def analyze_engagement_patterns(datasets):
    """Examines student engagement in vle, displays patterns in interaction frequency and timing."""

    vle_data = datasets['vle_interactions']

    print("\nEngagement Pattern Analysis")
    print("="*50)

    # overall engagement metrics
    print("\nDaily Interaction Summary:")
    daily_interactions = vle_data.groupby('id_student')['sum_click'].sum()
    print(daily_interactions.describe())

    # activity timing
    print("\nTemporal Distribution of Activities:")
    activity_timing = vle_data.groupby('date')['sum_click'].mean()
    print(activity_timing.describe())

In [12]:
def run_eda_pipeline(datasets):
    """Executes a complete EDA pipeline, displaying insights about data quality, student performance, and engagement patterns directly in the console."""

    print("\nStarting EDA...")
    perform_automated_eda(datasets)
    analyze_student_performance(datasets)
    analyze_engagement_patterns(datasets)

    print("\nEDA Complete.")

In [13]:
def document_eda_findings(datasets):
    """Documents key findings from EDA to inform modeling decisions."""
    
    findings = {
        "demographic_insights": [],
        "performance_patterns": [],
        "engagement_patterns": [],
        "data_quality_issues": [],
        "modeling_implications": []
    }
    
    # demographic insights
    student_data = datasets['student_info']
    findings["demographic_insights"].append(
        f"Age distribution: {student_data['age_band'].value_counts(normalize=True).to_dict()}"
    )
    findings["demographic_insights"].append(
        f"Gender distribution: {student_data['gender'].value_counts(normalize=True).to_dict()}"
    )
    findings["demographic_insights"].append(
        f"IMD band distribution: {student_data['imd_band'].value_counts(normalize=True).to_dict()}"
    )
    
    # performance patterns
    if 'student_assessments' in datasets:
        performance_data = datasets['student_info'].merge(
            datasets['student_assessments'],
            on='id_student'
        )
        
        # age-performance relationship
        age_perf = performance_data.groupby('age_band', observed=False)['score'].mean().to_dict()
        findings["performance_patterns"].append(f"Age-performance relationship: {age_perf}")
        
        # imd-performance relationship
        imd_perf = performance_data.groupby('imd_band', observed=False)['score'].mean().to_dict()
        findings["performance_patterns"].append(f"IMD-performance relationship: {imd_perf}")
    
    # engagement patterns
    if 'vle_interactions' in datasets:
        vle_data = datasets['vle_interactions']
        
        # activity distribution over time
        activity_time = vle_data.groupby('date')['sum_click'].mean()
        findings["engagement_patterns"].append(
            f"Peak engagement day: Day {activity_time.idxmax()} with {activity_time.max():.2f} avg clicks"
        )
        
        # student engagement variability
        student_engagement = vle_data.groupby('id_student')['sum_click'].sum()
        findings["engagement_patterns"].append(
            f"Engagement variability: min={student_engagement.min()}, max={student_engagement.max()}, " 
            f"median={student_engagement.median()}, std={student_engagement.std():.2f}"
        )
    
    # data quality issues
    for name, df in datasets.items():
        missing = df.isnull().sum()
        if missing.any():
            findings["data_quality_issues"].append(
                f"{name} has missing values: {missing[missing > 0].to_dict()}"
            )
    
    # modeling implications
    findings["modeling_implications"].append(
        "Age shows clear correlation with performance; age-specific models might be beneficial"
    )
    findings["modeling_implications"].append(
        "IMD band shows socioeconomic gradient in performance; fairness metrics needed"
    )
    findings["modeling_implications"].append(
        "High variability in engagement suggests temporal features will be critical"
    )
    
    return findings

## **Cleaning and Validation**

In [14]:
def clean_demographic_data(student_info: pd.DataFrame) -> pd.DataFrame:
    """Performs initial cleaning of demographic data, handles missing values, and standardizes formats."""

    cleaned_data = student_info.copy()

    # standardize string columns
    string_columns = ['gender', 'region', 'highest_education', 'imd_band', 'age_band']
    for col in string_columns:
        cleaned_data[col] = cleaned_data[col].str.strip().str.lower()

    # handle missing values
    cleaned_data['imd_band'] = cleaned_data['imd_band'].fillna('unknown')
    cleaned_data['disability'] = cleaned_data['disability'].fillna('N')

    return cleaned_data

In [15]:
def clean_vle_data(vle_interactions: pd.DataFrame,
                   vle_materials: pd.DataFrame) -> pd.DataFrame:
    """Cleans and merges VLE interaction data, removes outliers/invalid entries."""

    # remove interactions with invalid click counts
    cleaned_interactions = vle_interactions[vle_interactions['sum_click'] > 0]

    # merge with materials data
    merged_data = cleaned_interactions.merge(
        vle_materials,
        on=['id_site', 'code_module', 'code_presentation'],
        how='left'
    )

    # remove entries with missing material types
    merged_data = merged_data.dropna(subset=['activity_type'])

    return merged_data

In [16]:
def clean_assessment_data(assessments: pd.DataFrame,
                         student_assessments: pd.DataFrame) -> pd.DataFrame:
    """Cleans assessment data, handling missing scores and invalid dates."""

    # remove invalid scores
    valid_assessments = student_assessments[
        (student_assessments['score'] >= 0) &
        (student_assessments['score'] <= 100)
    ]

    # merge with assessment data
    cleaned_data = valid_assessments.merge(
        assessments,
        on='id_assessment',
        how='left'
    )

    return cleaned_data

In [17]:
def validate_data_consistency(datasets: Dict[str, pd.DataFrame]) -> bool:
    """Validates consistency across datasets."""

    try:
        # check student IDs consistency across files
        student_ids = set(datasets['student_info']['id_student'])
        vle_ids = set(datasets['vle_interactions']['id_student'])
        assessment_ids = set(datasets['student_assessments']['id_student'])

        # ensure all students in VLE and assessments exist in student_info
        if not (vle_ids.issubset(student_ids) and assessment_ids.issubset(student_ids)):
            print("Warning: Some VLE or assessment records have unknown student IDs")

        # check module-presentation pairs consistency
        modules_presentations = set(zip(datasets['courses']['code_module'],
                                     datasets['courses']['code_presentation']))
        student_modules = set(zip(datasets['student_info']['code_module'],
                                datasets['student_info']['code_presentation']))

        if not student_modules.issubset(modules_presentations):
            print("Warning: Invalid module-presentation combinations found")

        return True

    except Exception as e:
        print(f"Validation failed: {str(e)}")
        return False

## **Processing Pipeline**

In [18]:
# load and validate datasets
datasets = load_raw_datasets()
if not validate_data_consistency(datasets):
    raise ValueError("Data validation failed. Check the datasets")

Loaded studentInfo.csv: 32593 rows, 11 columns
Loaded studentVle.csv: 10655280 rows, 6 columns
Loaded vle.csv: 6364 rows, 6 columns
Loaded assessments.csv: 206 rows, 6 columns
Loaded studentAssessment.csv: 173912 rows, 5 columns
Loaded courses.csv: 22 rows, 3 columns

Total rows loaded across all files: 10,868,377


In [19]:
# eda analysis
run_eda_pipeline(datasets)


Starting EDA...

Analysis of student_info

Dataset Overview:
Number of rows: 32593
Number of columns: 11

Columns and their data types:
code_module             category
code_presentation       category
id_student                 int32
gender                  category
region                  category
highest_education       category
imd_band                category
age_band                category
num_of_prev_attempts        int8
studied_credits            int64
disability              category
dtype: object

Missing Values Analysis:
imd_band    1111
dtype: int64

Numerical Columns Summary:
       studied_credits
count     32593.000000
mean         79.758691
std          41.071900
min          30.000000
25%          60.000000
50%          60.000000
75%         120.000000
max         655.000000

Analysis of vle_interactions

Dataset Overview:
Number of rows: 10655280
Number of columns: 6

Columns and their data types:
code_module          category
code_presentation    category
id_studen

In [20]:
# document findings from eda for later reference
eda_findings = document_eda_findings(datasets)

# print structured eda findings
print("\n=== DOCUMENTED EDA FINDINGS ===")
for category, items in eda_findings.items():
    print(f"\n{category.upper()}")
    print("-" * len(category))
    for item in items:
        print(f"• {item}")


=== DOCUMENTED EDA FINDINGS ===

DEMOGRAPHIC_INSIGHTS
--------------------
• Age distribution: {'0-35': 0.7039548369281747, '35-55': 0.2894179731844261, '55<=': 0.0066271898873991346}
• Gender distribution: {'M': 0.5484306446169422, 'F': 0.4515693553830577}
• IMD band distribution: {'20-30%': 0.11606632361349342, '30-40%': 0.11241344260212184, '10-20': 0.11168286639984754, '0-10%': 0.10517120894479386, '40-50%': 0.103424178895877, '50-60%': 0.0992313067784766, '60-70%': 0.0922749507655168, '70-80%': 0.09144908201511975, '80-90%': 0.08773267263833301, '90-100%': 0.08055396734642017}

PERFORMANCE_PATTERNS
--------------------
• Age-performance relationship: {'0-35': 74.55190257051278, '35-55': 77.25301108206502, '55<=': 79.57217165149545}
• IMD-performance relationship: {'0-10%': 72.1047655904467, '10-20': 73.33945278915694, '20-30%': 74.30418701692825, '30-40%': 74.88306611025698, '40-50%': 75.35335069618351, '50-60%': 75.51088425161387, '60-70%': 75.89883973894126, '70-80%': 75.838699

In [21]:
# clean datasets
cleaned_demographics = clean_demographic_data(datasets['student_info'])
cleaned_vle = clean_vle_data(datasets['vle_interactions'],
                            datasets['vle_materials'])
cleaned_assessments = clean_assessment_data(datasets['assessments'],
                                          datasets['student_assessments'])

# **II. Feature Engineering**

## **Feature Creation**

In [22]:
def create_demographic_features(cleaned_demographics: pd.DataFrame) -> pd.DataFrame:
    """Transforms cleaned demographic data into model features. Preserves key identifier columns before transformations."""

    features = cleaned_demographics.copy()

    # preserve key identifier columns
    id_columns = ['id_student', 'code_module', 'code_presentation']
    preserved_ids = features[id_columns].copy()

    # create encoded categorical variables
    categorical_columns = ['gender', 'region', 'highest_education',
                         'imd_band', 'age_band']

    for col in categorical_columns:
        # label encoding
        features[f"{col}_encoded"] = pd.Categorical(features[col]).codes

        # one-hot encoding for tree-based models
        one_hot = pd.get_dummies(features[col],
                                prefix=col,
                                drop_first=True)
        features = pd.concat([features, one_hot], axis=1)

    # create educational background features
    features['is_first_attempt'] = (features['num_of_prev_attempts'] == 0)
    features['credit_density'] = features['studied_credits'] / features['num_of_prev_attempts'].clip(1)

    # use preserved key column copies
    for col in id_columns:
        features[col] = preserved_ids[col]

    return features

In [23]:
def create_temporal_features(cleaned_vle_data: pd.DataFrame,
                           window_sizes: List[int]) -> Dict[str, pd.DataFrame]:
    """Creates time-based engagement features from VLE data using multiple window sizes."""

    temporal_features = {}

    for window_size in window_sizes:
        # create time windows
        cleaned_vle_data['window'] = cleaned_vle_data['date'] // window_size

        # store grouping columns
        group_cols = ['id_student', 'code_module', 'code_presentation', 'window']

        # handle numeric aggregations
        numeric_metrics = cleaned_vle_data.groupby(group_cols).agg({
            'sum_click': ['sum', 'mean', 'std'],
            'id_site': 'nunique'
        })

        # flatten column names
        numeric_metrics.columns = [
            f"{col[0]}_{col[1]}" if isinstance(col, tuple) else col
            for col in numeric_metrics.columns
        ]
        numeric_metrics = numeric_metrics.reset_index()

        # handle activity type aggregation
        activity_counts = pd.pivot_table(
            cleaned_vle_data,
            index=group_cols,
            columns='activity_type',
            values='sum_click',
            aggfunc='count',
            fill_value=0
        )

        # rename activity columns and reset index
        activity_counts.columns = [f'activity_{col}' for col in activity_counts.columns]
        activity_counts = activity_counts.reset_index()

        # merge dataframes
        temporal_features[f"window_{window_size}"] = numeric_metrics.merge(
            activity_counts,
            on=group_cols,
            validate='one_to_one'  # validates 1-1 merge
        )

    return temporal_features

In [24]:
def create_assessment_features(cleaned_assessment_data: pd.DataFrame) -> pd.DataFrame:
    """Creates assessment-based features."""

    # preserve key identifier structure
    group_cols = ['id_student', 'code_module', 'code_presentation']

    # calculate submission delay and weighted components
    cleaned_assessment_data['submission_delay'] = (
        cleaned_assessment_data['date_submitted'] - cleaned_assessment_data['date']
    )
    cleaned_assessment_data['score_weight_product'] = (
        cleaned_assessment_data['score'] * cleaned_assessment_data['weight']
    )

    # group and aggregate all metrics
    performance_metrics = cleaned_assessment_data.groupby(group_cols).agg({
        'score': ['mean', 'std', 'min', 'max', 'count'],
        'submission_delay': ['mean', 'std'],
        'weight': 'sum',
        'is_banked': 'sum',
        'score_weight_product': 'sum'
    }).reset_index()  # preserves original column names

    # flatten column names except identifier columns unchanged
    performance_metrics.columns = [
        col[0] if col[0] in group_cols else f"{col[0]}_{col[1]}"
        for col in performance_metrics.columns
    ]

    # calculate final metrics
    performance_metrics['weighted_score'] = (
        performance_metrics['score_weight_product_sum'] /
        performance_metrics['weight_sum'].replace(0, np.nan)
    ).fillna(0)

    performance_metrics['submission_consistency'] = (
        performance_metrics['submission_delay_std'] /
        performance_metrics['submission_delay_mean'].replace(0, np.nan)
    ).fillna(0)

    # drop intermediate calculation columns
    performance_metrics = performance_metrics.drop(
        ['score_weight_product_sum', 'weight_sum'],
        axis=1
    )

    return performance_metrics

In [25]:
def create_sequential_features(cleaned_vle_data: pd.DataFrame) -> pd.DataFrame:
    """Creates sequential features for the GRU/LSTM path, maintaining temporal ordering crucial for sequential models."""

    # sort by student and time
    sequential_data = cleaned_vle_data.sort_values(['id_student', 'date'])

    # create time-based features
    sequential_data['time_since_last'] = sequential_data.groupby('id_student')['date'].diff()
    sequential_data['cumulative_clicks'] = sequential_data.groupby('id_student')['sum_click'].cumsum()

    # create activity transition features
    sequential_data['prev_activity'] = sequential_data.groupby('id_student')['activity_type'].shift()

    return sequential_data

In [26]:
def prepare_dual_path_features(demographic_features, temporal_features,
                             assessment_features, sequential_features,
                             chunk_size: int = 50000):
    """Prepares dual-path features with chunked processing."""

    # process static path in chunks
    static_chunks = []
    for chunk_start in range(0, len(demographic_features), chunk_size):
        chunk_end = chunk_start + chunk_size
        demo_chunk = demographic_features.iloc[chunk_start:chunk_end]

        # merge chunk with assessment features
        static_chunk = demo_chunk.merge(
            assessment_features,
            on=['id_student', 'code_module', 'code_presentation'],
            how='inner'
        )
        static_chunks.append(static_chunk)

    # combine static path chunks
    static_path = pd.concat(static_chunks, ignore_index=True)

    # process sequential path similarly
    sequential_path = sequential_features.merge(
        temporal_features['window_7'],
        on=['id_student', 'code_module', 'code_presentation', 'window'],
        how='inner'
    )

    return {
        'static_path': static_path,
        'sequential_path': sequential_path
    }

In [27]:
# create all features
demographic_features = create_demographic_features(cleaned_demographics)
temporal_features = create_temporal_features(cleaned_vle, window_sizes=[7, 14, 30])
assessment_features = create_assessment_features(cleaned_assessments)
sequential_features = create_sequential_features(cleaned_vle)

In [28]:
dual_path_features = prepare_dual_path_features(
    demographic_features,
    temporal_features,
    assessment_features,
    sequential_features,
    chunk_size=50000
)

In [29]:
# print features to console
print("\nStatic Path Features:")
print(f"Number of samples: {len(dual_path_features['static_path'])}")
print("Features included:", dual_path_features['static_path'].columns.tolist())

print("\nSequential Path Features:")
print(f"Number of samples: {len(dual_path_features['sequential_path'])}")
print("Features included:", dual_path_features['sequential_path'].columns.tolist())

# set static and sequential features
static_features = dual_path_features['static_path']
sequential_features = dual_path_features['sequential_path']

# verify our data alignment
print("\nData Alignment Check:")
print(f"Number of unique students in static path: {static_features['id_student'].nunique()}")
print(f"Number of unique students in sequential path: {sequential_features['id_student'].nunique()}")


Static Path Features:
Number of samples: 25820
Features included: ['code_module', 'code_presentation', 'id_student', 'gender', 'region', 'highest_education', 'imd_band', 'age_band', 'num_of_prev_attempts', 'studied_credits', 'disability', 'gender_encoded', 'gender_m', 'region_encoded', 'region_east midlands region', 'region_ireland', 'region_london region', 'region_north region', 'region_north western region', 'region_scotland', 'region_south east region', 'region_south region', 'region_south west region', 'region_wales', 'region_west midlands region', 'region_yorkshire region', 'highest_education_encoded', 'highest_education_he qualification', 'highest_education_lower than a level', 'highest_education_no formal quals', 'highest_education_post graduate qualification', 'imd_band_encoded', 'imd_band_10-20', 'imd_band_20-30%', 'imd_band_30-40%', 'imd_band_40-50%', 'imd_band_50-60%', 'imd_band_60-70%', 'imd_band_70-80%', 'imd_band_80-90%', 'imd_band_90-100%', 'imd_band_unknown', 'age_band

## **Dataset Splitting**

In [30]:
def create_stratified_splits(dual_path_features, test_size=0.2, random_state=0):
    """Creates stratified train/test splits preserving demographic distributions."""
    from sklearn.model_selection import train_test_split
    
    # extract static path features for stratification
    static_features = dual_path_features['static_path']
    
    # create stratification columns
    static_features['strat_gender'] = static_features['gender']
    static_features['strat_age'] = static_features['age_band']
    static_features['strat_imd'] = static_features['imd_band'].apply(
        lambda x: x if pd.notna(x) else 'unknown'
    )
    
    # create combined stratification column
    static_features['stratify_col'] = static_features['strat_gender'] + '_' + \
                                     static_features['strat_age'] + '_' + \
                                     static_features['strat_imd'].astype(str)
    
    # get unique student ids
    all_student_ids = static_features['id_student'].unique()
    
    # create student-level stratified split
    student_df = static_features[['id_student', 'stratify_col']].drop_duplicates()
    
    # perform stratified split
    train_ids, test_ids = train_test_split(
        student_df['id_student'],
        test_size=test_size,
        random_state=random_state,
        stratify=student_df['stratify_col']
    )
    
    # split static and sequential features
    static_train = static_features[static_features['id_student'].isin(train_ids)]
    static_test = static_features[static_features['id_student'].isin(test_ids)]
    
    sequential_train = dual_path_features['sequential_path'][
        dual_path_features['sequential_path']['id_student'].isin(train_ids)
    ]
    sequential_test = dual_path_features['sequential_path'][
        dual_path_features['sequential_path']['id_student'].isin(test_ids)
    ]
    
    # verify stratification maintained demographic proportions
    for col in ['gender', 'age_band', 'imd_band']:
        print(f"\nDistribution of {col} in training set:")
        print(static_train[col].value_counts(normalize=True))
        
        print(f"\nDistribution of {col} in test set:")
        print(static_test[col].value_counts(normalize=True))
    
    return {
        'static_train': static_train,
        'static_test': static_test,
        'sequential_train': sequential_train,
        'sequential_test': sequential_test,
        'train_ids': train_ids,
        'test_ids': test_ids
    }

In [31]:
# create train/test splits with stratification across demographic features
print("\nCreating stratified train/test splits...")
data_splits = create_stratified_splits(dual_path_features, test_size=0.2, random_state=0)

# access the splits
static_train = data_splits['static_train']
static_test = data_splits['static_test']
sequential_train = data_splits['sequential_train']
sequential_test = data_splits['sequential_test']

# print split sizes
print(f"\nSplit sizes:")
print(f"Static train: {len(static_train)} samples, {static_train['id_student'].nunique()} students")
print(f"Static test: {len(static_test)} samples, {static_test['id_student'].nunique()} students")
print(f"Sequential train: {len(sequential_train)} samples")
print(f"Sequential test: {len(sequential_test)} samples")

# save the train/test split IDs for future reference
train_ids = data_splits['train_ids']
test_ids = data_splits['test_ids']


Creating stratified train/test splits...

Distribution of gender in training set:
gender
m    0.549799
f    0.450201
Name: proportion, dtype: float64

Distribution of gender in test set:
gender
m    0.553138
f    0.446862
Name: proportion, dtype: float64

Distribution of age_band in training set:
age_band
0-35     0.694336
35-55    0.298409
55<=     0.007256
Name: proportion, dtype: float64

Distribution of age_band in test set:
age_band
0-35     0.694070
35-55    0.298999
55<=     0.006931
Name: proportion, dtype: float64

Distribution of imd_band in training set:
imd_band
30-40%     0.107870
20-30%     0.106177
10-20      0.100808
40-50%     0.098679
50-60%     0.098534
0-10%      0.093213
60-70%     0.092633
70-80%     0.092439
80-90%     0.088134
90-100%    0.082717
unknown    0.038795
Name: proportion, dtype: float64

Distribution of imd_band in test set:
imd_band
20-30%     0.107239
30-40%     0.106854
10-20      0.100308
40-50%     0.099538
50-60%     0.098575
0-10%      0.0951

## **Feature Validation**

In [33]:
def prepare_target_variable(data):
    """Creates binary target variable from final_result column."""
    
    # final result mapping (0: not at risk, 1: at risk)
    risk_mapping = {
        'pass': 0,
        'distinction': 0,
        'fail': 1,
        'withdrawal': 1
    }
    
    # convert to lowercase and map to binary target
    return data['final_result'].str.lower().map(risk_mapping)

In [ ]:
def analyze_feature_importance(X, y, n_estimators=100, random_state=0):
    """Analyzes feature importance using Random Forest classifier."""
        
    # select numeric columns for importance analysis
    numeric_cols = X.select_dtypes(include=['int64', 'float64', 'bool']).columns
    X_numeric = X[numeric_cols]
    
    # train random forest classifier
    model = RandomForestClassifier(n_estimators=n_estimators, random_state=random_state)
    model.fit(X_numeric, y)
    importances = model.feature_importances_ # feature importances
    
    # convert to df for visualization
    feature_importance = pd.DataFrame({
        'Feature': numeric_cols,
        'Importance': importances
    })
    feature_importance = feature_importance.sort_values('Importance', ascending=False) # sort by importance
    
    # plot feature importances
    plt.figure(figsize=(12, 8))
    plt.barh(feature_importance['Feature'], feature_importance['Importance'])
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.title('Feature Importance from Random Forest')
    plt.tight_layout()
    plt.show()
    
    return feature_importance

In [ ]:
def analyze_feature_correlations(X, threshold=0.85):
    """Analyzes correlation between numeric features."""
    
    # select numeric columns
    numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
    X_numeric = X[numeric_cols]
    
    # calculate correlation matrix
    corr_matrix = X_numeric.corr()
    
    # plot correlation heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', center=0)
    plt.title('Feature Correlation Matrix')
    plt.tight_layout()
    plt.show()
    
    # iterate through correlation matrix to find highly correlated pairs
    correlated_pairs = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                correlated_pairs.append({
                    'Feature1': corr_matrix.columns[i],
                    'Feature2': corr_matrix.columns[j],
                    'Correlation': corr_matrix.iloc[i, j]
                })
    
    # convert to df
    if correlated_pairs:
        return pd.DataFrame(correlated_pairs)
    else:
        return pd.DataFrame(columns=['Feature1', 'Feature2', 'Correlation'])

In [ ]:
# merge in final_result column if not already in static_features
if 'final_result' not in static_features.columns:
    print("Debug step, if shown")
    static_features = static_features.merge(
        datasets['student_info'][['id_student', 'code_module', 'code_presentation', 'final_result']],
        on=['id_student', 'code_module', 'code_presentation'],
        how='left'
    )
else:
    print("DEBUG STATEMENT: Delete if statement in final_result column.")

# prepare target variable
y = prepare_target_variable(static_features)

# Create feature matrix (exclude non-feature columns)
exclude_cols = ['id_student', 'code_module', 'code_presentation', 'final_result', 
                'stratify_col', 'strat_gender', 'strat_age', 'strat_imd']
X = static_features.drop(columns=[col for col in exclude_cols if col in static_features.columns])

# Analyze feature importance
importance_df = analyze_feature_importance(X, y, n_estimators=100)
print("Top 10 most important features:")
print(importance_df.head(10))

# Analyze feature correlations
corr_df = analyze_feature_correlations(X, threshold=0.85)
if not corr_df.empty:
    print("\nHighly correlated feature pairs:")
    print(corr_df)
else:
    print("\nNo feature pairs with correlation above threshold were found.")

# Identify low-importance features
low_importance_features = importance_df[importance_df['Importance'] < 0.01]['Feature'].tolist()
if low_importance_features:
    print(f"\nLow importance features (below 0.01):")
    print(low_importance_features)
    print("\nConsider removing these features to simplify the model.")